### Import libraries and get response code

In [2]:
import requests
import pandas as pd 
import numpy as np
from bs4 import BeautifulSoup

In [3]:
# Send a request to the page
url = 'https://www.reed.co.uk/jobs/data-analyst-jobs'
response = requests.get(url)

In [4]:
# Download html with a get request
response = requests.get(url)
response.status_code # 200 status code means OK!

200

### Scrape job listings (reed.co.uk)

In [5]:
# Create an empty list
jobs = []

# Loop through all pages
for page in range(1, 160): 
    url = f'https://www.reed.co.uk/jobs/data-analyst-jobs?pageno={page}'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Extract job details
    job_listings = soup.select('div.row > div.col-xs-12.col-sm-8.col-md-9.details')
    for job in job_listings:
        # Extract the job title
        title = job.find('h2').text.strip()

        # Extract the company name
        company = job.find('a', {'class': 'gtmJobListingPostedBy'}).text.strip()

        # Extract the location (if available)
        location_elem = job.find('li', {'class': 'job-metadata__item--location'})
        location = location_elem.text.strip() if location_elem else 'Not specified'

        # Extract the salary (if available)
        salary_elem = job.find('li', {'class': 'job-metadata__item--salary'})
        salary = salary_elem.text.strip() if salary_elem else 'Not specified'

        # Extract job type
        jobtype = job.find('li', {'class': 'job-metadata__item--type'}).text.strip()

        # Extract job pref
        remote_elem = job.find('li', {'class': 'job-metadata__item--remote'})
        remote = remote_elem.text.strip() if remote_elem else 'Not specified'

        # Extract the job description
        job_link = 'https://www.reed.co.uk' + job.find('a', href=True)['href']
        job_response = requests.get(job_link)
        job_soup = BeautifulSoup(job_response.content, 'html.parser')
        job_description = job_soup.find('span', {'itemprop': 'description'})
        if job_description:
            job_description = job_description.text.strip()
        else:
            job_description = 'No description available'
            
        # Append the job information
        jobs.append([title, company, location, salary, jobtype, remote, job_description, job_link])
        

In [6]:
# Create a dataframe from the list of jobs
columns = ['title','company','location','salary','type','remote','description','link']
jobs_df = pd.DataFrame(jobs, columns=columns)

In [7]:
jobs_df.shape

(4293, 8)

In [8]:
jobs_df

,title,company,location,salary,type,remote,description,link
0,Data Analyst,e.surv Chartered Surveyors,Kettering\r\n Northamptonshire,Salary negotiable,"Permanent, full-time",Work from home,"Trading since 1989, e.surv Chartered Surveyors...",https://www.reed.co.uk/jobs/data-analyst/49988...
1,Data Analyst,University of the West of Scotland,Paisley\r\n Renfrewshire,"£32,000 - £39,000 per annum","Contract, full-time",Not specified,"Registry Full time, Fixed-term for 24 months T...",https://www.reed.co.uk/jobs/data-analyst/50197...
2,Data Analyst,Brandon James Ltd,Bristol\r\n Avon,"£40,000 - £45,000 per annum","Contract, full-time",Not specified,A strategic medium sized consultancy is now se...,https://www.reed.co.uk/jobs/data-analyst/50020...
3,Data Analyst,Brandon James Ltd,London\r\n London,"£40,000 - £45,000 per annum","Contract, full-time",Not specified,A strategic medium sized consultancy is now se...,https://www.reed.co.uk/jobs/data-analyst/50020...
4,Data Analyst,The Consultancy Group,London\r\n London,"£35,000 - £50,000 per annum","Permanent, full-time",Not specified,I am partnered with a renowned Retail business...,https://www.reed.co.uk/jobs/data-analyst/50188...
...,...,...,...,...,...,...,...,...
4288,Cloud Platform Engineer,Hays Specialist Recruitment Limited,Ireland,"£40,000 - £50,000 per annum","Permanent, full-time",Not specified,Cloud Platform Engineer - Up to £50k - Hybrid ...,https://www.reed.co.uk/jobs/cloud-platform-eng...
4289,Quantitative Risk Management Analyst Consultant,eFinancialCareers,London\r\n London,Competitive salary,"Permanent, full-time",Not specified,DescriptionJob Description Assist the team in...,https://www.reed.co.uk/jobs/quantitative-risk-...
4290,Test Lead,Proactive Appointments,Windsor\r\n Berkshire,"£42,000 - £52,000 per annum","Permanent, full-time",Not specified,Test Lead Our client is seeking an experienc...,https://www.reed.co.uk/jobs/test-lead/50039066...
4291,Senior Team Lead Quant Analyst/ London,eFinancialCareers,London\r\n London,Competitive salary,"Permanent, full-time",Not specified,Role :- Your role will be to manage a small te...,https://www.reed.co.uk/jobs/senior-team-lead-q...


In [23]:
jobs_df.to_csv("jobs.csv", index=False)